# AdventureWorks – Försäljningsanalys

Denna notebook innehåller en analys av försäljningsdata från AdventureWorks-databasen.
Data hämtas via SQL och analyseras vidare i Python med Pandas och visualiseras med Matplotlib.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

server = "localhost,1433"
database = "AdventureWorks2025"
username = "sa"
password = quote_plus("Str0ngPass!2025")
driver = quote_plus("ODBC Driver 18 for SQL Server")

connection_string = (
    f"mssql+pyodbc://{username}:{password}@{server}/{database}"
    f"?driver={driver}&Encrypt=yes&TrustServerCertificate=yes"
)

engine = create_engine(connection_string)

In [ ]:
with engine.connect() as conn:
    print(conn.execute(text("SELECT DB_NAME()")).scalar())

In [ ]:
def read_sql_df(query: str) -> pd.DataFrame:
    with engine.connect() as conn:
        return pd.read_sql(text(query), conn)

## 1. Antal produkter per kategori

In [ ]:
query_1 = """
SELECT
    pc.Name AS Category,
    COUNT(DISTINCT p.ProductID) AS ProductCount
FROM Production.ProductCategory pc
JOIN Production.ProductSubcategory psc
    ON psc.ProductCategoryID = pc.ProductCategoryID
JOIN Production.Product p
    ON p.ProductSubcategoryID = psc.ProductSubcategoryID
GROUP BY pc.Name
ORDER BY ProductCount DESC;
"""

df_1 = read_sql_df(query_1)
df_1

In [ ]:
plt.figure()
plt.bar(df_1["Category"], df_1["ProductCount"])
plt.title("Antal produkter per kategori")
plt.xlabel("Kategori")
plt.ylabel("Antal produkter")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


In [ ]:
most = df_1.iloc[0]
least = df_1.iloc[-1]
most["Category"], int(most["ProductCount"]), least["Category"], int(least["ProductCount"])


**Insikt:** Kategorien med flest produkter är **Bikes** med **99** produkter.
Kategorien med minst produkter är **Clothing** med **25** produkter.

## 2. Försäljning per produktkategori

In [ ]:
query_2 = """
SELECT
    pc.Name AS Category,
    SUM(sod.LineTotal) AS TotalSales
FROM Sales.SalesOrderDetail sod
JOIN Production.Product p
    ON sod.ProductID = p.ProductID
JOIN Production.ProductSubcategory psc
    ON p.ProductSubcategoryID = psc.ProductSubcategoryID
JOIN Production.ProductCategory pc
    ON psc.ProductCategoryID = pc.ProductCategoryID
GROUP BY pc.Name
ORDER BY TotalSales DESC;
"""

df_2 = read_sql_df(query_2)
df_2

In [ ]:
df_2_plot = df_2.sort_values("TotalSales", ascending=True)

plt.figure()
plt.barh(df_2_plot["Category"], df_2_plot["TotalSales"])
plt.title("Total försäljning per produktkategori")
plt.xlabel("Total försäljning")
plt.ylabel("Kategori")
plt.tight_layout()
plt.show()

In [ ]:
highest = df_2.iloc[0]
lowest = df_2.iloc[-1]

highest["Category"], round(highest["TotalSales"], 2), lowest["Category"], round(lowest["TotalSales"], 2)

**Insikt:** Den kategori som genererat högst total försäljning är **"Bikes"**.
Den kategori som genererat lägst total försäljning är **"Accessories"**.

## 3. Försäljningstrend per månad

In [ ]:
query_3 = """
SELECT
    DATEFROMPARTS(YEAR(OrderDate), MONTH(OrderDate), 1) AS MonthStart,
    SUM(TotalDue) AS TotalSales
FROM Sales.SalesOrderHeader
GROUP BY DATEFROMPARTS(YEAR(OrderDate), MONTH(OrderDate), 1)
ORDER BY MonthStart;
"""

df_3 = read_sql_df(query_3)
df_3["MonthStart"] = pd.to_datetime(df_3["MonthStart"])
df_3

In [ ]:
plt.figure()
plt.plot(df_3["MonthStart"], df_3["TotalSales"])
plt.title("Försäljningstrend per månad")
plt.xlabel("Månad")
plt.ylabel("Total försäljning")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
highest_month = df_3.loc[df_3["TotalSales"].idxmax()]
lowest_month = df_3.loc[df_3["TotalSales"].idxmin()]

highest_month["MonthStart"].strftime("%Y-%m"), round(float(highest_month["TotalSales"]), 2), \
lowest_month["MonthStart"].strftime("%Y-%m"), round(float(lowest_month["TotalSales"]), 2)


**Insikt:** Försäljningen över tid varierar tydligt mellan olika månader.
Den högsta försäljningen sker under **2025-04** med **5 847 164,69**, medan den lägsta försäljningen sker under **2025-06** med **52 478,19**.

## 4. Antal ordrar och total försäljning per år


In [ ]:
query_4 = """
SELECT
    YEAR(OrderDate) AS OrderYear,
    COUNT(SalesOrderID) AS OrderCount,
    SUM(TotalDue) AS TotalSales
FROM Sales.SalesOrderHeader
GROUP BY YEAR(OrderDate)
ORDER BY OrderYear;
"""

df_4 = read_sql_df(query_4)
df_4


In [ ]:
plt.figure()

x = range(len(df_4))

plt.bar(x, df_4["OrderCount"], label="Antal ordrar")
plt.bar(x, df_4["TotalSales"], label="Total försäljning")

plt.xticks(x, df_4["OrderYear"])
plt.title("Antal ordrar och total försäljning per år")
plt.xlabel("År")
plt.ylabel("Värde")
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
most_orders = df_4.loc[df_4["OrderCount"].idxmax()]
most_sales = df_4.loc[df_4["TotalSales"].idxmax()]

most_orders["OrderYear"], int(most_orders["OrderCount"]), \
most_sales["OrderYear"], round(float(most_sales["TotalSales"]), 2)


**Insikt:** Flest ordrar lades under **2024** med **14 244** ordrar.
Den högsta totala försäljningen uppnåddes också under **2024** med **49 020 486,51**.

## 5. Topp 10 produkter baserat på försäljning


In [ ]:
query_5 = """
SELECT TOP 10
    p.Name AS ProductName,
    SUM(sod.LineTotal) AS TotalSales
FROM Sales.SalesOrderDetail sod
JOIN Production.Product p
    ON sod.ProductID = p.ProductID
GROUP BY p.Name
ORDER BY TotalSales DESC;
"""

df_5 = read_sql_df(query_5)
df_5


In [ ]:
df_5_plot = df_5.sort_values("TotalSales", ascending=True)

plt.figure()
plt.barh(df_5_plot["ProductName"], df_5_plot["TotalSales"])
plt.title("Topp 10 produkter baserat på försäljning")
plt.xlabel("Total försäljning")
plt.ylabel("Produkt")
plt.tight_layout()
plt.show()


In [ ]:
top_product = df_5.iloc[0]

top_product["ProductName"], round(float(top_product["TotalSales"]), 2)


**Insikt:** Den produkt som genererat högst total försäljning är **Mountain-200 Black** med **4400592.8** i försäljning.

## 6. Försäljning och antal kunder per region

In [ ]:
query_6 = """
SELECT
    cr.Name AS Region,
    COUNT(DISTINCT soh.CustomerID) AS CustomerCount,
    SUM(soh.TotalDue) AS TotalSales
FROM Sales.SalesOrderHeader soh
JOIN Sales.SalesTerritory st
    ON soh.TerritoryID = st.TerritoryID
JOIN Person.CountryRegion cr
    ON st.CountryRegionCode = cr.CountryRegionCode
GROUP BY cr.Name
ORDER BY TotalSales DESC;
"""

df_6 = read_sql_df(query_6)
df_6


In [ ]:
df_6_plot = df_6.sort_values("TotalSales", ascending=True)

plt.figure()
plt.barh(df_6_plot["Region"], df_6_plot["TotalSales"])
plt.title("Total försäljning per region")
plt.xlabel("Total försäljning")
plt.ylabel("Region")
plt.tight_layout()
plt.show()


In [ ]:
top_region = df_6.iloc[0]
bottom_region = df_6.iloc[-1]

top_region["Region"], round(float(top_region["TotalSales"]), 2), \
bottom_region["Region"], round(float(bottom_region["TotalSales"]), 2)


**Insikt:** Regionen med högst total försäljning är **United States** med **70829863.2**.
Regionen med lägst total försäljning är **Germany** med **5479819.58**.


## 7. Genomsnittligt ordervärde per region och kundtyp


In [ ]:
query_7 = """
WITH base AS (
    SELECT
        st.Name AS Region,
        CASE 
            WHEN s.BusinessEntityID IS NOT NULL THEN 'Store'
            ELSE 'Individual'
        END AS CustomerType,
        COUNT(*) AS OrderCount,
        SUM(soh.TotalDue) AS TotalSales,
        SUM(soh.TotalDue) / COUNT(*) AS AvgOrderValue
    FROM Sales.SalesOrderHeader soh
    JOIN Sales.Customer c
        ON soh.CustomerID = c.CustomerID
    LEFT JOIN Sales.Store s
        ON c.StoreID = s.BusinessEntityID
    JOIN Sales.SalesTerritory st
        ON soh.TerritoryID = st.TerritoryID
    GROUP BY
        st.Name,
        CASE 
            WHEN s.BusinessEntityID IS NOT NULL THEN 'Store'
            ELSE 'Individual'
        END
),
region_order AS (
    SELECT
        Region,
        SUM(TotalSales) / SUM(OrderCount) AS RegionAvgOrderValue
    FROM base
    GROUP BY Region
)
SELECT
    b.Region,
    b.CustomerType,
    b.OrderCount,
    b.TotalSales,
    b.AvgOrderValue,
    ro.RegionAvgOrderValue
FROM base b
JOIN region_order ro
    ON b.Region = ro.Region
ORDER BY ro.RegionAvgOrderValue DESC, b.CustomerType;
"""

df_7 = read_sql_df(query_7)
df_7


In [ ]:
# Pivot: en rad per region, en kolumn per kundtyp
pivot_7 = df_7.pivot_table(
    index="Region",
    columns="CustomerType",
    values="AvgOrderValue",
    aggfunc="mean"
)

# Sortera regioner efter total genomsniitt 
region_order = (
    df_7.drop_duplicates(subset=["Region"])
       .set_index("Region")["RegionAvgOrderValue"]
       .sort_values(ascending=False)
)

pivot_7 = pivot_7.loc[region_order.index]
pivot_7


In [ ]:
import numpy as np

regions = pivot_7.index.tolist()
x = np.arange(len(regions))
width = 0.4

plt.figure()

# Om någon kolumn saknas i data, fyll med 0 för att undvika fel
individual_vals = pivot_7["Individual"] if "Individual" in pivot_7.columns else pd.Series([0]*len(regions), index=regions)
store_vals = pivot_7["Store"] if "Store" in pivot_7.columns else pd.Series([0]*len(regions), index=regions)

plt.bar(x - width/2, individual_vals.values, width, label="Individual")
plt.bar(x + width/2, store_vals.values, width, label="Store")

plt.title("Genomsnittligt ordervärde per region och kundtyp")
plt.xlabel("Region")
plt.ylabel("Genomsnittligt ordervärde")
plt.xticks(x, regions, rotation=45, ha="right")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Högst/lägst region+kundtyp
temp = df_7[["Region", "CustomerType", "AvgOrderValue"]].copy()

highest = temp.loc[temp["AvgOrderValue"].idxmax()]
lowest = temp.loc[temp["AvgOrderValue"].idxmin()]

# Jämförelse Store vs Individual totalt (genomsnitt över regioner, vägt med ordercount)
weighted = df_7.copy()
weighted["WeightedAvg"] = weighted["AvgOrderValue"] * weighted["OrderCount"]

type_summary = (weighted.groupby("CustomerType")[["WeightedAvg", "OrderCount"]].sum())
type_summary["OverallAvgOrderValue"] = type_summary["WeightedAvg"] / type_summary["OrderCount"]

highest["Region"], highest["CustomerType"], round(float(highest["AvgOrderValue"]), 2), \
lowest["Region"], lowest["CustomerType"], round(float(lowest["AvgOrderValue"]), 2), \
type_summary["OverallAvgOrderValue"].round(2).to_dict()


**Insikt:** Högst genomsnittligt ordervärde har **Southwest** för **Store-kunder** med **27 739,06**.
Lägst genomsnittligt ordervärde har **Central** för **Individual-kunder** med **368,44**.

Vid jämförelse mellan kundtyper visar sammanställningen att **Store-kunder** har ett betydligt högre genomsnittligt ordervärde än **Individual-kunder**.

## Djupanalys: Skillnader mellan Store- och Individual-kunder

I denna del analyseras skillnaader i försäljningsbeteende mellan Store ochh Individual-kunder.
Analysen fokuserar på ordervolym, försäljning och genomsnittligt ordervärde över tid och mellan regioner.


In [ ]:
query_d1 = """
SELECT
    YEAR(soh.OrderDate) AS OrderYear,
    CASE 
        WHEN s.BusinessEntityID IS NOT NULL THEN 'Store'
        ELSE 'Individual'
    END AS CustomerType,
    COUNT(*) AS OrderCount,
    SUM(soh.TotalDue) AS TotalSales
FROM Sales.SalesOrderHeader soh
JOIN Sales.Customer c
    ON soh.CustomerID = c.CustomerID
LEFT JOIN Sales.Store s
    ON c.StoreID = s.BusinessEntityID
GROUP BY
    YEAR(soh.OrderDate),
    CASE 
        WHEN s.BusinessEntityID IS NOT NULL THEN 'Store'
        ELSE 'Individual'
    END
ORDER BY OrderYear, CustomerType;
"""

df_d1 = read_sql_df(query_d1)
df_d1


In [ ]:
pivot_orders = df_d1.pivot_table(
    index="OrderYear",
    columns="CustomerType",
    values="OrderCount",
    aggfunc="sum"
)

pivot_sales = df_d1.pivot_table(
    index="OrderYear",
    columns="CustomerType",
    values="TotalSales",
    aggfunc="sum"
)

pivot_orders, pivot_sales


In [ ]:
plt.figure()
plt.plot(pivot_orders.index, pivot_orders["Individual"], label="Individual")
plt.plot(pivot_orders.index, pivot_orders["Store"], label="Store")

plt.title("Orderutveckling per kundtyp över tid")
plt.xlabel("År")
plt.ylabel("Antal ordrar")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
plt.figure()
plt.plot(pivot_sales.index, pivot_sales["Individual"], label="Individual")
plt.plot(pivot_sales.index, pivot_sales["Store"], label="Store")

plt.title("Försäljningsutveckling per kundtyp över tid")
plt.xlabel("År")
plt.ylabel("Total försäljning")
plt.legend()
plt.tight_layout()
plt.show()


**Reflektion:** Analysen visar tydliga skillnader mellan Store- och Individual-kunder.
Store-kunder står för en betydligt högre total försäljning trots färre ordrar,
vilket indikerar ett högre genomsnittligt ordervärde.

Individual-kunder har fler ordrar men med lägre värde per order.
Över tid är mönstret stabilt, vilket tyder på att kundtyperna har olika köpbeteenden snarare än tillfälliga variationer.
